In [0]:
%load_ext autoreload
%autoreload 2

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
ratings = pd.read_csv("/content/drive/My Drive/ML/myanimelist/animelists_reduced.csv")


In [4]:
ratings.shape

(159469, 11)

In [0]:
users= pd.read_csv("/content/drive/My Drive/ML/myanimelist/users_reduced.csv")

In [0]:
animes= pd.read_csv("/content/drive/My Drive/ML/myanimelist/animes_reduced.csv")

In [7]:
ratings.head(2)

,username,anime_id,my_watched_episodes,my_start_date,my_finish_date,my_score,my_status,my_rewatching,my_rewatching_ep,my_last_updated,my_tags
0,karthiga,21,586,0000-00-00,0000-00-00,9,1,NaN,0,2013-03-03 10:52:53,NaN
1,karthiga,59,26,0000-00-00,0000-00-00,7,2,NaN,0,2013-03-10 13:54:51,NaN


In [8]:
# Nos quedamos solo con su username y id
users.drop(columns = ['user_watching','user_completed','user_onhold','user_dropped','user_plantowatch','user_days_spent_watching','gender','location',
                      'birth_date','access_rank','join_date','last_online','stats_mean_score','stats_rewatched','stats_episodes'], inplace = True)
users.head()  

,username,user_id
0,karthiga,2255153
1,Damonashu,37326
2,bskai,228342
3,terune_uzumaki,327311
4,Bas_G,5015094


In [9]:
#Nos quedamos con los atributos que nos interesan de cada Data Frame
ratings.drop(columns=['my_watched_episodes', 'my_start_date', 'my_finish_date', 'my_status', 'my_rewatching', 'my_rewatching_ep', 'my_last_updated', 'my_tags'], inplace = True)
ratings.head()

,username,anime_id,my_score
0,karthiga,21,9
1,karthiga,59,7
2,karthiga,74,7
3,karthiga,120,7
4,karthiga,178,7


In [10]:
# Nos quedamos con los siguiente atributos en anime
# anime_id	title	type	source	episodes	score	scored_by genre
animes = animes.drop(columns = ['title_english', 'title_japanese', 'title_synonyms', 'image_url', 'status', 'airing', 'aired_string', 'aired', 'duration', 'rating', 'rank','popularity', 'members']) 
animes = animes.drop(columns = ['favorites', 'background', 'premiered', 'broadcast', 'related', 'producer', 'licensor', 'studio', 'opening_theme', 'ending_theme', 'duration_min', 'aired_from_year'])
animes.head()

,anime_id,title,type,source,episodes,score,scored_by,genre
0,11013,Inu x Boku SS,TV,Manga,12,7.63,139250,"Comedy, Supernatural, Romance, Shounen"
1,2104,Seto no Hanayome,TV,Manga,26,7.89,91206,"Comedy, Parody, Romance, School, Shounen"
2,5262,Shugo Chara!! Doki,TV,Manga,51,7.55,37129,"Comedy, Magic, School, Shoujo"
3,721,Princess Tutu,TV,Original,38,8.21,36501,"Comedy, Drama, Magic, Romance, Fantasy"
4,12365,Bakuman. 3rd Season,TV,Manga,25,8.67,107767,"Comedy, Drama, Romance, Shounen"


In [11]:
# Filtramos los animes que solo son de TV
anime_tv = animes[animes['type']=='TV']
anime_tv.head()

,anime_id,title,type,source,episodes,score,scored_by,genre
0,11013,Inu x Boku SS,TV,Manga,12,7.63,139250,"Comedy, Supernatural, Romance, Shounen"
1,2104,Seto no Hanayome,TV,Manga,26,7.89,91206,"Comedy, Parody, Romance, School, Shounen"
2,5262,Shugo Chara!! Doki,TV,Manga,51,7.55,37129,"Comedy, Magic, School, Shoujo"
3,721,Princess Tutu,TV,Original,38,8.21,36501,"Comedy, Drama, Magic, Romance, Fantasy"
4,12365,Bakuman. 3rd Season,TV,Manga,25,8.67,107767,"Comedy, Drama, Romance, Shounen"


In [12]:
# Solo cambiamos el nombre de la columna my_score por comodidad
ratings.rename(columns= {'my_score' : 'score'}, inplace = True)
ratings.head()

,username,anime_id,score
0,karthiga,21,9
1,karthiga,59,7
2,karthiga,74,7
3,karthiga,120,7
4,karthiga,178,7


In [13]:
# Unimos los dos Data Frames
merged = ratings.merge(anime_tv, left_on = 'anime_id', right_on = 'anime_id', suffixes= ['_user', ''])
merged.head()

,username,anime_id,score_user,title,type,source,episodes,score,scored_by,genre
0,karthiga,21,9,One Piece,TV,Manga,0,8.54,423868,"Action, Adventure, Comedy, Super Power, Drama,..."
1,Damonashu,21,10,One Piece,TV,Manga,0,8.54,423868,"Action, Adventure, Comedy, Super Power, Drama,..."
2,bskai,21,8,One Piece,TV,Manga,0,8.54,423868,"Action, Adventure, Comedy, Super Power, Drama,..."
3,Slimak,21,10,One Piece,TV,Manga,0,8.54,423868,"Action, Adventure, Comedy, Super Power, Drama,..."
4,MistButterfly,21,0,One Piece,TV,Manga,0,8.54,423868,"Action, Adventure, Comedy, Super Power, Drama,..."


In [14]:
merged.shape

(111837, 10)

In [0]:
# Nos quedamos con 4*10^4 usuarios para agilizar los calculos
merged = merged[['username', 'title', 'score_user']]
merged = merged.merge(users, left_on = 'username', right_on = 'username')
merged.head()

,username,title,score_user,user_id
0,karthiga,One Piece,9,2255153
1,karthiga,Chobits,7,2255153
2,karthiga,Gakuen Alice,7,2255153
3,karthiga,Fruits Basket,7,2255153
4,karthiga,Ultra Maniac,7,2255153


In [0]:
merged.shape

(43135, 4)

In [0]:
# Experimentalmente con 10^4 datos no da RTE
data = merged

# COMENTAR LA LINEA DE ABAJO PARA USAR TODOS LOS DATOS DISPONIBLES
#data = data[data.user_id <= 10000]
data.drop(columns = ['username'], inplace = True)
data.head()

,title,score_user,user_id
0,One Piece,9,2255153
1,Chobits,7,2255153
2,Gakuen Alice,7,2255153
3,Fruits Basket,7,2255153
4,Ultra Maniac,7,2255153


In [0]:
piv = data.pivot_table(index=['user_id'], columns=['title'], values='score_user')

In [0]:
print(piv.shape)
piv.head()

(199, 2401)


title,.hack//Roots,.hack//Sign,.hack//Tasogare no Udewa Densetsu,009-1,07-Ghost,11eyes,12-sai.: Chicchana Mune no Tokimeki,18if,25-sai no Joshikousei,3-gatsu no Lion,...,Zoku Touken Ranbu: Hanamaru,Zombie-Loan,"Zone of the Enders: Dolores, I",ef: A Tale of Melodies.,ef: A Tale of Memories.,iDOLM@STER Xenoglossia,s.CRY.ed,xxxHOLiC,xxxHOLiC Kei,ēlDLIVE
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,7.0,7.0,NaN,NaN
935,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN
3895,NaN,NaN,NaN,NaN,9.0,10.0,NaN,NaN,NaN,8.0,...,NaN,8.0,NaN,10.0,9.0,NaN,NaN,9.0,9.0,NaN
4266,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,5.0,6.0,NaN,NaN,NaN,NaN,NaN
4463,NaN,NaN,NaN,7.0,10.0,10.0,NaN,NaN,NaN,NaN,...,NaN,5.0,NaN,NaN,NaN,NaN,8.0,10.0,0.0,NaN


In [0]:
import numpy as np
# Normalizamos los puntajes de cada usuario z = (x - u) / sigma
piv_norm = piv.apply(lambda x: (x-np.mean(x))/(np.max(x)-np.min(x)), axis=1)

# Quitamos todas las columnas con solo ceros ya que son animes que no han sido puntuados por nadie
piv_norm.fillna(0, inplace=True)
piv_norm = piv_norm.T
piv_norm = piv_norm.loc[:, (piv_norm != 0).any(axis=0)]

In [0]:
values=piv.values

In [0]:
values.shape[1]

2401

In [0]:
values = values.astype('float32') / 10

In [0]:
values

array([[nan, 0.7, nan, ..., 0.7, nan, nan],
       [nan, nan, nan, ..., 1. , nan, nan],
       [nan, nan, nan, ..., 0.9, 0.9, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]], dtype=float32)

In [0]:
train= values[:150]

In [0]:
test= values[150:]

In [0]:
from keras.layers import Input, Dense
from keras.models import Model

# this is the size of our encoded representations
encoding_dim = 512  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

# this is our input placeholder
input_Animes = Input(shape=(train.shape[1],))
# "encoded" is the encoded representation of the input
encoded = Dense(encoding_dim, activation='relu')(input_Animes)
# "decoded" is the lossy reconstruction of the input
decoded = Dense(values.shape[1], activation='sigmoid')(encoded)

# this model maps an input to its reconstruction
autoencoder = Model(input_Animes, decoded)

In [0]:
# this model maps an input to its encoded representation
encoder = Model(input_Animes, encoded)

In [0]:
# create a placeholder for an encoded (32-dimensional) input
encoded_input = Input(shape=(encoding_dim,))
# retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# create the decoder model
decoder = Model(encoded_input, decoder_layer(encoded_input))

In [0]:
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

In [0]:
autoencoder.fit(train, train,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(test, test))

Train on 150 samples, validate on 49 samples
Epoch 1/50
150/150 [==============================] - 2s 17ms/step - loss: nan - val_loss: nan
Epoch 2/50
150/150 [==============================] - 0s 137us/step - loss: nan - val_loss: nan
Epoch 3/50
150/150 [==============================] - 0s 112us/step - loss: nan - val_loss: nan
Epoch 4/50
150/150 [==============================] - 0s 114us/step - loss: nan - val_loss: nan
Epoch 5/50
150/150 [==============================] - 0s 114us/step - loss: nan - val_loss: nan
Epoch 6/50
150/150 [==============================] - 0s 96us/step - loss: nan - val_loss: nan
Epoch 7/50
150/150 [==============================] - 0s 104us/step - loss: nan - val_loss: nan
Epoch 8/50
150/150 [==============================] - 0s 102us/step - loss: nan - val_loss: nan
Epoch 9/50
150/150 [==============================] - 0s 104us/step - loss: nan - val_loss: nan
Epoch 10/50
150/150 [==============================] - 0s 109us/step - loss: nan - val_loss: 